<a href="https://colab.research.google.com/github/PierreSylvain/active-learning/blob/main/train_std.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Active learning
https://huggingface.co/docs/transformers/tasks/sequence_classification



In [ ]:
!pip install -q huggingface_hub
!pip install -q transformers datasets evaluate
!pip install -q accelerate -U

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def tokenize(examples):
  return tokenizer(examples["text"], truncation=True, padding=True)

In [ ]:
imdb_base = load_dataset("imdb")
display(imdb_base)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imdb_base['train']['text'], imdb_base['train']['label'],
                                                    test_size=0.20,
                                                    random_state=63)
train_dataset = Dataset.from_dict({
    'text': X_train,
    'label': y_train
})

test_dataset = Dataset.from_dict({
    'text': X_test,
    'label': y_test
})

# Creating the DatasetDict
imdb = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})
print(imdb)

In [ ]:
tokenized_imdb = imdb.map(tokenize, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
eval_dataset = imdb_base["test"]
eval_dataset = eval_dataset.map(tokenize, batched=True, batch_size=len(eval_dataset))

In [ ]:
eval_result = trainer.evaluate(eval_dataset=eval_dataset)
print(eval_result)